# DLC-style visualization of tracked points

## First: what format does DLC expect?

In [1]:

import os
import deeplabcut
from DLC_for_WBFM.bin.configuration_definition import *
import pandas as pd
from DLC_for_WBFM.utils.video_and_data_conversion.video_conversion_utils import write_minimax_projection_from_btf
import numpy as np
from tqdm import tqdm
import deeplabcut
from DLC_for_WBFM.utils.preprocessing.DLC_utils import create_dlc_project_from_config
from DLC_for_WBFM.utils.preprocessing.convert_matlab_annotations_to_DLC import csv_annotations2config_names
from DLC_for_WBFM.utils.feature_detection.visualize_using_dlc import *
%load_ext autoreload
%autoreload 2


In [2]:
# Get the format for a DLC annotation
proj_folder = r'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02'
annotations = r'videos\test2020-10-22_mcherry_fr1000_sl10_15DLC_resnet50_wbfm_3dNov2shuffle1_200000.h5'

fname = os.path.join(proj_folder, annotations)


In [14]:
dlc_df = pd.read_hdf(fname)

In [15]:
# list(dlc_df.columns)

In [16]:
#dlc_df

## Import tracklets and write in this format

In [58]:
#df

In [60]:
#new_dlc_df

# Make a raw and labeled .avi video

## Use a config file to organize everything

In [11]:
# Define the project folder
project_folder = r'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment'

# Initialize the config files
task_name = 'feature_alignment'
experimenter = 'Charlie'

config = DLCForWBFMConfig(task_name=task_name, experimenter=experimenter)

# Note: Windows-style filename
# working_directory = '.'

config = create_project(config, project_folder)

Created Project folder "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19"


In [12]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

red_avi_fname = None
green_avi_fname = None

# Create the first sub-object
# This remembers the location of the raw data
datafiles = DLCForWBFMDatafiles(btf_fname_red, btf_fname_green, red_avi_fname, green_avi_fname)

config.datafiles = datafiles
save_config(config)

In [13]:
print(config)

                Field values:
                task_name: feature_alignment 
                experimenter: Charlie 
                config_filename: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19\config.pickle
                Which subclasses are initialized?
                datafiles: True
                preprocessing: False
                tracking: False
                traces: False



In [16]:
# Set up options for the .avi video
# opt = {'start_volume':           0, 
#        'num_frames':             500, 
#        'num_total_slices':       33, 
#        'num_crop_slices':        11, 
#        'center_slice':           12, 
#        'alpha':                  0.15,
#        'fps':                    5.0,
#        'red_and_green_mirrored': True}
opt = {'start_volume':           50, 
       'num_frames':             500, 
       'num_total_slices':       33, 
       'num_crop_slices':        11, 
       'center_slice':           14, 
       'alpha':                  0.15,
       'fps':                    5.0,
       'red_and_green_mirrored': True}
preprocessing = DLCForWBFMPreprocessing(**opt)

config.preprocessing = preprocessing
save_config(config)

In [17]:
# Actually write the RAW video

config = write_minimax_projection_from_btf(config)

Converting volume index 50 to frame index 1650
Converting volume index 50 to frame index 1650


## Use the new DLC folder to hold everything

i.e. import the tracklets, and write in DLC format

NOTE: MOVE THE PICKLE FILES INTO THIS FOLDER MANUALLY (for now)

In [14]:
project_folder = config.get_dirname()

In [4]:
#fname = os.path.join(project_folder, 'clust_df_dat.pickle')
# fname = os.path.join(project_folder, 'clust_df_dat_minimax.pickle')
fname = os.path.join(project_folder, 'stitched_df_dat.pickle')
clust_df = pd.read_pickle(fname)

In [5]:
new_dlc_df = build_dlc_annotation_all(clust_df, min_length=50, num_frames=500, verbose=0)

100%|████████████████████████████████████████████████████████████████████████████| 8518/8518 [00:02<00:00, 3189.75it/s]


In [15]:
#scorer = 'feature_tracker'
scorer = 'feature_tracker_stitched'

# output_path = os.path.join(project_folder)
new_dlc_df.to_csv(os.path.join(project_folder,"CollectedData_" + scorer + ".csv"))
new_dlc_df.to_hdf(os.path.join(project_folder,"CollectedData_" + scorer + ".h5"), key="df_with_missing", mode="w")
new_dlc_df.to_pickle(os.path.join(project_folder,"CollectedData_" + scorer + ".pickle"))
# deeplabcut.convertcsv2h5(path_config_file, userfeedback=False)

## Now, the annotations themselves

In [3]:
######
# Only if restarting
# fname = os.path.join('feature_alignment-Charlie-2021-01-06','config.pickle')
# config = load_config(fname)

In [17]:
# Save the annotations in the config file
project_folder = config.get_dirname()

#annotation_fname = 'CollectedData_feature_tracker.h5'
annotation_fname = 'CollectedData_feature_tracker_stitched.h5'
annotation_fname = os.path.join(project_folder, annotation_fname)

vid_fname = r'redfr1000_sl7_17.avi'
vid_fname = os.path.join(project_folder, vid_fname)

tracking = DLCForWBFMTracking(project_folder, vid_fname, annotation_fname)

config.tracking = tracking
save_config(config)

## Finally, label the video using the config file

### Only create a project if needed

In [20]:
dlc_config = create_dlc_project_from_config(config, '1')

Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19\f1-C-2021-02-19\videos"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19\f1-C-2021-02-19\labeled-data"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19\f1-C-2021-02-19\training-datasets"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19\f1-C-2021-02-19\dlc-models"
Copying the videos
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-02-19\f1-C-2021-02-19\videos\redfr500_sl5_15.avi
Generated "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\ex

In [31]:
# IF NOT DEFINED ABOVE
dlc_config = config.tracking.DLC_config_fname
dlc_config

'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-06\\f1-C-2021-01-06\\config.yaml'

In [39]:
#config

In [12]:
#bugfix
#dlc_config = os.path.join('C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-06\\f1-C-2021-01-06', 'config.yaml')
#dlc_config = os.path.join(dlc_config, 'config.yaml')
#config.tracking.DLC_config_fname = dlc_config
#save_config(config)

In [19]:
# Make sure the dlc config file has all the neuron names
csv_annotations = config.tracking.annotation_fname.replace('h5', 'csv')
csv_annotations2config_names(dlc_config, csv_annotations)

Adding body part annotations for 429 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written


In [22]:
# Use my wrapper function
project_folder = config.get_dirname()

video_fname = config.datafiles.red_avi_fname

#scorer = 'feature_tracker'
scorer = 'feature_tracker_stitched'
df = pd.read_hdf(os.path.join(project_folder,"CollectedData_" + scorer + ".h5"), key="df_with_missing")


In [28]:
make_labeled_video_custom_annotations(dlc_config, video_fname, df)

  2%|█▊                                                                               | 11/500 [00:00<00:05, 90.06it/s]

Duration of video [s]: 50.0, recorded with 10.0 fps!
Overall # of frames: 500 with cropped frame dimensions: 900 700
Generating frames and creating video.


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:06<00:00, 77.57it/s]


# Do this using a unified function

## First, assuming the raw video and DLC folder is already made

In [20]:
project_folder = 'feature_alignment-Charlie-2021-01-22'

# Overall project settings, including link to video
config_fname = 'config.pickle'
config_fname = os.path.join(project_folder, config_fname)
# The tracks, saved as a dataframe
# df_fname = 'clust_df_dat_affine.pickle'
# df_fname = 'clust_df_dat_prealigned.pickle'
# df_fname = 'clust_df_dat_affine_nominimax.pickle'
# df_fname = 'clust_df_dat_prealigned-stitched.pickle'
df_fname = 'clust_df_dat_affine-and-features.pickle'
# df_fname = 'clust_df_dat_affine-and-features-stitched.pickle'
# df_fname = 'clust_df_dat_prealigned-stitched100-2.pickle'
# df_fname = 'clust_df_dat_sequential7.pickle'

opt = {'total_num_frames':500, 'min_track_length':300}

In [21]:
new_dlc_df = create_video_from_annotations(config_fname, df_fname, **opt)

  0%|                                                                       | 0/500 [00:00<?, ?it/s]

Found 500 tracks of length >300
Adding body part annotations for 5 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written
Duration of video [s]: 50.0, recorded with 10.0 fps!
Overall # of frames: 500 with cropped frame dimensions: 900 700
Generating frames and creating video.


100%|████████████████████████████████████████████████████████████| 500/500 [00:04<00:00, 109.04it/s]


### OTHER FOLDER, for starting at volume 50


In [3]:
project_folder = 'feature_alignment-Charlie-2021-02-19'
config_fname = 'config.pickle'
config_fname = os.path.join(project_folder, config_fname)

# df_fname = 'clust_df_dat_sequential7.pickle'
# df_fname = 'clust_df_dat_sequential19-gp.pickle'
# df_fname = 'clust_df_dat_sequential19.pickle'
# df_fname = 'clust_df_dat_sequential7-fullbipartite.pickle'
# df_fname = 'clust_df_dat_sequential-stardist.pickle'
# df_fname = 'clust_df_dat_reference-11.pickle'
# df_fname = 'clust_df_dat_reference-31.pickle'
df_fname = 'clust_df_dat_bipartite-stardist.pickle'


opt = {'total_num_frames':500, 'min_track_length':400}

In [4]:
new_dlc_df = create_video_from_annotations(config_fname, df_fname, **opt)

100%|███████████████████████████████████████████████████████████████████████████| 3001/3001 [00:00<00:00, 13712.67it/s]

Found no tracks long enough; aborting


In [5]:
new_dlc_df

In [4]:
c = load_config(config_fname)
c

DLCForWBFMConfig(task_name='feature_alignment', experimenter='Charlie', datafiles=DLCForWBFMDatafiles(red_bigtiff_fname='D:\\More-stabilized-wbfm\\test2020-10-22_16-15-20_test4-channel-0-pco_camera1\\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf', green_bigtiff_fname='D:\\More-stabilized-wbfm\\test2020-10-22_16-15-20_test4-channel-1-pco_camera2\\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf', red_avi_fname='C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-02-19\\redfr500_sl5_15.avi', green_avi_fname='C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-22\\greenfr500_sl7_17.avi'), preprocessing=DLCForWBFMPreprocessing(start_volume=0, num_frames=500, fps=5.0, alpha=0.15, num_total_slices=33, num_crop_slices=11, center_slice=12, red_and_green_mirrored=True), tracking=D

In [33]:
# Fix folders of this config file
# c.datafiles.red_avi_fname = c.datafiles.red_avi_fname.replace('2021-01-22', '2021-02-19')
c.datafiles.red_avi_fname = 'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-02-19\\redfr500_sl5_15.avi'
c.tracking.DLC_config_fname = c.tracking.DLC_config_fname.replace('2021-01-22', '2021-02-19')
c.config_filename = c.config_filename.replace('2021-01-22', '2021-02-19')

save_config(c)

### Create one file per neuron


In [17]:
out = create_many_videos_from_annotations(config_fname, df_fname, **opt, num_videos_to_make=10)

  2%|█▉                                                                              | 12/500 [00:00<00:04, 111.05it/s]

Adding body part annotations for 2 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written
Duration of video [s]: 50.0, recorded with 10.0 fps!
Overall # of frames: 500 with cropped frame dimensions: 900 700
Generating frames and creating video.


  2%|█▊                                                                              | 11/500 [00:00<00:04, 104.38it/s]

Adding body part annotations for 2 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written
Duration of video [s]: 50.0, recorded with 10.0 fps!
Overall # of frames: 500 with cropped frame dimensions: 900 700
Generating frames and creating video.


  2%|█▌                                                                               | 10/500 [00:00<00:05, 93.31it/s]

...

Adding body part annotations for 2 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written
Duration of video [s]: 50.0, recorded with 10.0 fps!
Overall # of frames: 500 with cropped frame dimensions: 900 700
Generating frames and creating video.


  2%|█▍                                                                                | 9/500 [00:00<00:05, 88.21it/s]

Adding body part annotations for 2 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written
Duration of video [s]: 50.0, recorded with 10.0 fps!
Overall # of frames: 500 with cropped frame dimensions: 900 700
Generating frames and creating video.


100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:04<00:00, 109.18it/s]

Finished making videos for 10 neurons


In [52]:

c = load_config(config_fname)
project_folder = c.get_dirname()

# Load the dataframe name, and produce DLC-style annotations
with open(df_fname, 'rb') as f:
    clust_df = pickle.load(f)
opt = {'min_length':450, 'num_frames':500, 'verbose':0}
new_dlc_df = build_dlc_annotation_all(clust_df, **opt)

100%|██████████████████████████████████████████████████████████████████████| 3862/3862 [00:00<00:00, 11475.51it/s]


In [31]:

c = load_config(config_fname)
project_folder = c.get_dirname()
project_folder

'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-22'